# This notebook applies wick's theorem to an arbitrary combination of potentials, raising and lowering operators, and exports the resulting diagrams to latex

In [1]:
import sympy
import numpy as np
import re
import matplotlib.pyplot as plt
import networkx as nx

### First, we define the functions for producing the all graphs and calculating the distinct ones: 

In [2]:
#this is for some reason much quicker, simple recursive implementation
def raw_determinant(matrix):
    if matrix.rows == 1:
        return matrix[0,0]
    return sum(matrix[0,i]*raw_determinant(matrix.minor_submatrix(0,i))*(1-2*(i%2)) 
               for i in range(matrix.rows))

In [3]:
# parses and applies wick's theorem to an expression which is a combination of 
# \psi(a), \psi^\dagger(a) and V(a-b), with a and b being arbitrary symbols.
# note: the sign will be wrong
def apply_wick(expr):
    operators = re.findall(r"\\psi\^?(\\dagger)?\(([^\)]*)\)", expr)
    creators = [x[1] for x in operators if x[0] != ""]
    anihalators = [x[1] for x in operators if x[0] == ""]
    matrix = sympy.Matrix(len(creators), len(anihalators), 
                          lambda i,j: sympy.symbols(f"G({anihalators[j]}-{creators[i]})"))
    
    potentials = re.findall(r"V\(([^)]*)-([^)]*)\)", expr)
    return raw_determinant(matrix).expand(), potentials

In [4]:
# iterates through the long sum that is the determinant, and creates a networkx
# graph for each. These graphs are then compared with each other for isomorphism,
# and a dictionary returning the multiplicity of each is returned
def generate_distinct_graphs(determinant, potentials):
    distinct_graphs = {}
    nm = nx.algorithms.isomorphism.categorical_node_match("type", "")
    for term in sympy.Add.make_args(determinant):
        G = nx.MultiDiGraph()
        for a, b in potentials:
            G.add_edge(a, b, type="photon")
            G.add_edge(b, a, type="photon")
        for term2 in sympy.Mul.make_args(term):
            if term2 == -1:
                continue
            a, b = re.search("\((.*)-(.*)\)", str(term2)).groups()
            G.add_edge(a, b, type="fermion")
            
        for G2 in distinct_graphs:
            if nx.is_isomorphic(G, G2, node_match=nm):
                distinct_graphs[G2] += 1
                break
        else:
            distinct_graphs[G.copy()] = 1
    return distinct_graphs

### Then we define the functions for exporting to latex (the tikz-feynman package is assumed)

In [5]:
# Utility function for plotting the graphs, not needed
def plot_graphs(G):
    for g in G:
        pos = nx.circular_layout(g)
        nx.draw_networkx_nodes(g, pos)
        nx.draw_networkx_labels(g, pos)
        photons = [(a, b) for (a, b, _), t in nx.get_edge_attributes(g,'type').items() if t=="photon"]
        electrons = [(a, b) for (a, b, _), t in nx.get_edge_attributes(g,'type').items() if t=="fermion"]
        nx.draw_networkx_edges(g, pos, photons, arrows=False, style="solid", width = 3, alpha=0.3)
        nx.draw_networkx_edges(g, pos, electrons, style="solid", connectionstyle="arc3,rad=0.1")
        plt.title(G[g])
        plt.show()

In [6]:
def get_header(g, potentials):
    pos = nx.circular_layout(g)
    return r"""
\tikzfeynmanset{every vertex={dot,minimum size=1mm}} 
\newcommand{\diag}[1]{
\begin{tikzpicture}
  \begin{feynman}
    """ + \
    "\n    ".join([f"\\vertex at ({round(x*1.5,2)}, {round(y*1.5,2)}) ({name});" for name, (x,y) in pos.items()]) + \
    """
    \diagram* {
      """ + \
    "\n      ".join([f"({a}) -- [photon] ({b})," for a,b in potentials]) +\
"""
      #1
    };
    \end{feynman}
\end{tikzpicture}
}"""

def get_diagram(g):
    string = "\diag{\n"
    for (a,b,_), species in nx.get_edge_attributes(g,'type').items():
        if species == "fermion":
            if a==b:
                string += f"      ({a}) -- [fermion, loop, min distance=1cm] {b},\n"
            else:
                string += f"      ({a}) -- [fermion, bend right] ({b}),\n"
    return string + "}"

def put_in_table(G, columns):
    string = "\\begin{center}\n\\begin{tabular}{"+ "|".join("c"*columns) + "}\n" 
    column = 0
    multiplicities = []
    for g in G:
        string += get_diagram(g)
        column += 1
        multiplicities.append(str(G[g]))
        if column == columns:
            string += "\\\\\n"
            column = 0
            string += "&".join(multiplicities)+r"\\\hline"
            multiplicities = []
            string += "\n"
        else:
            string += "&\n"
    if multiplicities != []:
        string += r"\\" + r"&".join(multiplicities)
    string += "\end{tabular}\n\end{center}"
    return string

Finally, now that all functionality is defined, we use it. 

In [7]:
S_expr = r"$\psi^\dagger(1)\psi^\dagger(1')V(1-1')\psi(1)\psi(1')$" +\
         r"$\psi^\dagger(2)\psi^\dagger(2')V(2-2')\psi(2)\psi(2')$"

In [8]:
determinant, potentials = apply_wick(S_expr)
G = generate_distinct_graphs(determinant, potentials)

print(get_header(list(G)[0], potentials))
print(put_in_table(G, 4))


\tikzfeynmanset{every vertex={dot,minimum size=1mm}} 
\newcommand{\diag}[1]{
\begin{tikzpicture}
  \begin{feynman}
    \vertex at (1.5, 0.0) (1);
    \vertex at (-0.0, 1.5) (1');
    \vertex at (-1.5, -0.0) (2);
    \vertex at (0.0, -1.5) (2');
    \diagram* {
      (1) -- [photon] (1'),
      (2) -- [photon] (2'),
      #1
    };
    \end{feynman}
\end{tikzpicture}
}
\begin{center}
\begin{tabular}{c|c|c|c}
\diag{
      (1) -- [fermion, loop, min distance=1cm] 1,
      (1') -- [fermion, loop, min distance=1cm] 1',
      (2) -- [fermion, loop, min distance=1cm] 2,
      (2') -- [fermion, loop, min distance=1cm] 2',
}&
\diag{
      (1) -- [fermion, bend right] (2'),
      (1') -- [fermion, loop, min distance=1cm] 1',
      (2) -- [fermion, bend right] (1),
      (2') -- [fermion, bend right] (2),
}&
\diag{
      (1) -- [fermion, bend right] (1'),
      (1') -- [fermion, bend right] (1),
      (2) -- [fermion, bend right] (2'),
      (2') -- [fermion, bend right] (2),
}&
\diag{
      (1)

In [9]:
determinant, potentials = apply_wick(S_expr + r"$\psi(a)\psi^\dagger(b)$")
G = generate_distinct_graphs(determinant, potentials)

print(get_header(list(G)[0], potentials))
print(put_in_table(G, 4))


\tikzfeynmanset{every vertex={dot,minimum size=1mm}} 
\newcommand{\diag}[1]{
\begin{tikzpicture}
  \begin{feynman}
    \vertex at (1.5, 0.0) (1);
    \vertex at (0.75, 1.3) (1');
    \vertex at (-0.75, 1.3) (2);
    \vertex at (-1.5, -0.0) (2');
    \vertex at (-0.75, -1.3) (a);
    \vertex at (0.75, -1.3) (b);
    \diagram* {
      (1) -- [photon] (1'),
      (2) -- [photon] (2'),
      #1
    };
    \end{feynman}
\end{tikzpicture}
}
\begin{center}
\begin{tabular}{c|c|c|c}
\diag{
      (1) -- [fermion, loop, min distance=1cm] 1,
      (1') -- [fermion, loop, min distance=1cm] 1',
      (2) -- [fermion, loop, min distance=1cm] 2,
      (2') -- [fermion, loop, min distance=1cm] 2',
      (a) -- [fermion, bend right] (b),
}&
\diag{
      (1) -- [fermion, loop, min distance=1cm] 1,
      (1') -- [fermion, loop, min distance=1cm] 1',
      (2) -- [fermion, bend right] (b),
      (2') -- [fermion, bend right] (2),
      (a) -- [fermion, bend right] (2'),
}&
\diag{
      (1) -- [fermion, be